In [2]:
!pwd

/Users/shihosato/src/github.com/twinkle13531/master_degree/202012/SA


In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
import SA_exact_test_functions as net
import pandas as pd
import numpy as np
import timeit

In [6]:
num_reads = 10000

In [7]:
bit  = 20
num = 3
path = '../input/ost{}_{}.csv'.format(bit, num)
df = pd.read_csv(path, sep=',')
bqm = net.make_Hamiltonian(df)
bqm

BinaryQuadraticModel({0: -39.0, 1: -32.0, 2: -28.0, 3: -28.0, 4: -39.0, 5: -28.0, 6: -39.0, 7: -21.0, 8: -39.0, 9: -28.0, 10: -21.0, 11: -32.0, 12: -39.0, 13: -39.0, 14: -21.0, 15: -32.0, 16: -32.0, 17: -32.0, 18: -39.0, 19: -32.0}, {(0, 1): 4, (0, 2): 4, (0, 3): 4, (0, 4): 6, (0, 5): 4, (0, 6): 6, (0, 7): 2, (0, 8): 6, (0, 9): 4, (0, 10): 2, (0, 11): 4, (0, 12): 6, (0, 13): 6, (0, 14): 2, (0, 15): 4, (0, 16): 4, (0, 17): 4, (0, 18): 6, (0, 19): 4, (1, 2): 2, (1, 3): 2, (1, 4): 4, (1, 5): 2, (1, 6): 4, (1, 7): 2, (1, 8): 4, (1, 9): 2, (1, 10): 2, (1, 11): 4, (1, 12): 4, (1, 13): 4, (1, 14): 2, (1, 15): 4, (1, 16): 4, (1, 17): 4, (1, 18): 4, (1, 19): 4, (2, 3): 4, (2, 4): 4, (2, 5): 4, (2, 6): 4, (2, 7): 2, (2, 8): 4, (2, 9): 4, (2, 10): 2, (2, 11): 2, (2, 12): 4, (2, 13): 4, (2, 14): 2, (2, 15): 2, (2, 16): 2, (2, 17): 2, (2, 18): 4, (2, 19): 2, (3, 4): 4, (3, 5): 4, (3, 6): 4, (3, 7): 2, (3, 8): 4, (3, 9): 4, (3, 10): 2, (3, 11): 2, (3, 12): 4, (3, 13): 4, (3, 14): 2, (3, 15): 2, (3, 

In [ ]:
def make_Hamiltonian(df, t1):
    t_list = calc_marginals(df)
    ## same_num_combi_list と combi_num_list は
    ## それぞれ (i, i) と　(i, j) (ただし i < j) を格納したリストになっていますが、もう少しシンプルな記法で作れそうです。
    ## サイズは確かにdf.shape[0]ですが、あらけじめNという変数に格納しておくと可読性が上がります。
    ## 変数は関数の中で使うときに混同がおこらなければもっと短くてかまいません。
    '''
    same_num_combi_list = [(i, i) for i in range(df.shape[0])]
    #print('same_num_combi_list:', same_num_combi_list)
    num_num_list = [str(n) for n in range(df.shape[0])]
    num_num = ''.join(num_num_list)
    #print('num_num:', num_num)
    combi_str = itertools.combinations(num_num, 2)#str
    combi_num_list = [tuple(map(int, i)) for i in (combi_str)]
    #print('combi_num_list:', combi_num_list)
    '''
    N=len(df)
    dup_list = [(i, i) for i in range(N)]
    comb_list = [(i, j) for i in range(N) for j in range(i+1, N)]
    #式
    #(y0+y1+y2+...-r0)**2
    #or
    #(y0*1+y1*0+y2*1+...-r1)**2
    ## Y, LI, SEX, AOP の4つに分けて作成しあとで統合するアプローチは正しいです。
    ## 変数が0, 1なので二乗しても変わらないことを考えると、
    ## lin[i] と quad[i, i] を分ける必要はなくて(linに統合してしまう)
    ## たとえばYの総和に関する制約は次のように表せます。
    ## Y #######
    # (Σy_i - r0)^2
    # = (Σy_i)^2 - 2 * r0 * (Σy_i) + r0^2
    # = Σ(y_i)^2 + 2Σ(y_i y_j) - 2 * r0 * (Σy_i) + r0^2   (ただし i < j)
    # = 2Σ(y_i y_j) + (1 - 2 * r0) * (Σy_i) + r0^2 (y_i * y_i = y_i を利用)
    lin_Y = [1-2*t_list[0] for (i, _) in dup_list] #同じy同士
    quad_Y = [2 for (i, j) in comb_list] #異なるy同士
    num_Y = t_list[0]**2 #数字の二乗
    ## LIに関しては式が少し煩雑になりますが書き下して、
    ## 次のように書けそうです。
    ## LI #######
    # (Σy_i LI_i - t1)^2
    # = (Σy_i LI_i)^2 - 2 * t1 * (Σy_i t_i) + t1^2
    # = Σ(y_i LI_i)^2 + 2Σ(y_i LI_i y_j LI_j) - 2 * t1 * (Σy_i LI_i) + t1^2   (ただし i < j)
    # = 2Σ(y_i LI_i y_j LI_j) + Σ ((LI_i - 2 * t1) y_i LI_i) + t1^2 (y_i * y_i = y_i を利用)
    # = 2Σ(LI_i LI_j y_i y_j) + Σ ((LI_i - 2 * t1) LI_i y_i) + t1^2
    LI = df['LI'].iloc
    lin_LI = [(LI[i] - 2 * t1) * LI[i] for (i, _) in dup_list]
    quad_LI = [2*LI[i] * LI[j] for (i, j) in comb_list]
    num_LI = t1**2
    ## 以下SEX, AOPも同様に展開して修正してみてください。
    ## ひとまず0でうめておきます。
    lin_SEX = lin_AOP = [0 for _ in dup_list]
    quad_SEX = quad_AOP = [0 for _ in comb_list]
    num_SEX = num_AOP = 0
    '''
    ## SEX #######
    lin_SEX = [-2*t_list[2] if df.iloc[i, 2]==1 else 0 for i in range(df.shape[0])]
    quad_SEX_1 = [1 if df.iloc[each_same_num_combi[0], 2]==1 else 0 for each_same_num_combi in (same_num_combi_list)]
    quad_SEX_2 = [2 if df.iloc[each_conbi[0], 2]==1 and df.iloc[each_conbi[1], 2]==1 else 0 for each_conbi in (combi_num_list)]
    num_SEX = t_list[2]**2
    #AOP
    lin_AOP = [-2*t_list[3] if df.iloc[i, 3]==1 else 0 for i in range(df.shape[0])]
    quad_AOP_1 = [1 if df.iloc[each_same_num_combi[0], 3]==1 else 0 for each_same_num_combi in (same_num_combi_list)]
    quad_AOP_2 = [2 if df.iloc[each_conbi[0], 3]==1 and df.iloc[each_conbi[1], 3]==1 else 0 for each_conbi in (combi_num_list)]
    num_AOP = t_list[3]**2
    '''
    ## zipは二つ以上のリストを引数にとることができるのと、
    ## リスト内包表記は、辞書をつくるときにも使えるので、
    ## コードを短くできます。
    ##統合作業
    #lin
    '''
    lin1 = [l1 + l2 for (l1, l2) in zip(lin_Y, lin_LI)]
    lin2 = [l1 + l2 for (l1, l2) in zip(lin_SEX, lin_AOP)]
    lin_list = [l1 + l2 for (l1, l2) in zip(lin1, lin2)]
    lin_list = lin_Y
    lin = {}
    for i, li in enumerate(lin_list):
        lin[i]=li
    '''
    lin_list = [sum(lin) for lin in zip(lin_Y, lin_LI, lin_SEX, lin_AOP)]
    lin = {i: lin_list[i] for (i, _) in dup_list}
    #quad_1
    '''
    quad1_1 = [q1 + q2 for (q1, q2) in zip(quad_Y_1, quad_LI_1)]
    quad1_2 = [q1 + q2 for (q1, q2) in zip(quad_SEX_1, quad_AOP_1)]
    quad_values1 = [q1 + q2 for (q1, q2) in zip(quad1_1, quad1_2)]#list
    quad = {}
    for i, each_same_num_combi in enumerate(same_num_combi_list):
        quad[each_same_num_combi] = quad_values1[i]
    #print('quad_ongoing:',quad)
    #quad_2
    quad2_1 = [q1 + q2 for (q1, q2) in zip(quad_Y_2, quad_LI_2)]
    #print('quad2_1:', quad2_1)
    quad2_2 = [q1 + q2 for (q1, q2) in zip(quad_SEX_2, quad_AOP_2)]
    quad_values2 = [q1 + q2 for (q1, q2) in zip(quad2_1, quad2_2)]#list
    #print('quad_values2:', quad_values2)
    for i, each_num_combi in enumerate(combi_num_list):
        quad[each_num_combi] = quad_values2[i]
    '''
    quad_values = [sum(quad) for quad in zip(quad_Y, quad_LI, quad_SEX, quad_AOP)]
    quad = {ij: quad_values[n] for (n, ij) in enumerate(comb_list)}
    #num
    num = num_Y + num_LI + num_SEX + num_AOP
    print('lin:', lin)
    print('quad:', quad)
    print('num:', num)
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY)#dic, dic, num

In [ ]:
def make_Hamiltonian(df):
    t_list = calc_marginals(df)
    
    N=len(df)
    SEX = df['SEX'].iloc
    AOP = df['AOP'].iloc
    
    dup_list = [(i, i) for i in range(N)]
    comb_list = [(i, j) for i in range(N) for j in range(i+1, N)]
    
    lin_Y = [1-2*t_list[0] for (i, _) in dup_list] #同じy同士
    lin_SEX  = [(SEX[i] - 2 * t_list[1]) * SEX[i] for (i, _) in dup_list]
    lin_AOP = [(AOP[i] - 2 * t_list[2]) * AOP[i] for (i, _) in dup_list]
    lin_list = [sum(lin) for lin in zip(lin_Y, lin_SEX, lin_AOP)]
    lin = {i: lin_list[i] for (i, _) in dup_list}
    
    quad_Y = [2 for (i, j) in comb_list] #異なるy同士
    quad_SEX  = [2*SEX[i] * SEX[j] for (i, j) in comb_list]
    quad_AOP = [2*AOP[i] * AOP[j] for (i, j) in comb_list]
    quad_values = [sum(quad) for quad in zip(quad_Y, quad_SEX, quad_AOP)]
    quad = {ij: quad_values[n] for (n, ij) in enumerate(comb_list)}
    
    num_Y = t_list[0]**2 #数字の二乗
    num_SEX  = t_list[1]**2
    num_AOP = t_list[2]**2
    num = num_Y + num_SEX + num_AOP
    
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY)#dic, dic, num

In [8]:
bit  = 20
num = 3
path = '../input/ost{}_{}.csv'.format(bit, num)
df = pd.read_csv(path, sep=',')

In [9]:
t_list = np.array(
        [sum(df['Y']),
         np.dot(df['Y'], df['SEX']),
         np.dot(df['Y'], df['AOP'])
		])

In [10]:
(Σy_i - r0)^2
= (Σy_i)^2 - 2 * r0 * (Σy_i) + r0^2
= Σ(y_i)^2 + 2Σ(y_i y_j) - 2 * r0 * (Σy_i) + r0^2   (ただし i < j)
= 2Σ(y_i y_j) + (1 - 2 * r0) * (Σy_i) + r0^2 (y_i * y_i = y_i を利用)

array([11,  6,  4])

In [30]:
quad_Y = [2 for (i, j) in comb_list] #異なるy同士
len(quad_Y)

190

$H = 2\sum_{p < q} y_py_q + (1-2 \hat{t}_0)\sum y_p + \hat{t}_0^2 + \sum_{j\neq k}(2\sum_{p < q}x_{pj}x_{qj}y_py_q + (1 -2\hat{t}_j)\sum y_px_{pj} + \hat{t}_j^2)$

In [13]:
dup_list = [(i, i) for i in range(N)]
dup_list

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10),
 (11, 11),
 (12, 12),
 (13, 13),
 (14, 14),
 (15, 15),
 (16, 16),
 (17, 17),
 (18, 18),
 (19, 19)]

In [31]:
comb_list = [(i, j) for i in range(N) for j in range(i+1, N)]
len(comb_list)

190

In [15]:
lin_Y = [1-2*t_list[0] for (i, _) in dup_list] #同じy同士
lin_Y

[-21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21,
 -21]

In [16]:
SEX = df['SEX'].iloc
lin_SEX  = [(SEX[i] - 2 * t_list[1]) * SEX[i] for (i, _) in dup_list]
lin_SEX

[-11,
 -11,
 0,
 0,
 -11,
 0,
 -11,
 0,
 -11,
 0,
 0,
 -11,
 -11,
 -11,
 0,
 -11,
 -11,
 -11,
 -11,
 -11]

In [17]:
AOP = df['AOP'].iloc
lin_AOP = [(AOP[i] - 2 * t_list[2]) * AOP[i] for (i, _) in dup_list]

In [18]:
lin_list = [sum(lin) for lin in zip(lin_Y, lin_SEX, lin_AOP)]
lin = {i: lin_list[i] for (i, _) in dup_list}

In [20]:
len(lin_list)

20

In [21]:
lin

{0: -39,
 1: -32,
 2: -28,
 3: -28,
 4: -39,
 5: -28,
 6: -39,
 7: -21,
 8: -39,
 9: -28,
 10: -21,
 11: -32,
 12: -39,
 13: -39,
 14: -21,
 15: -32,
 16: -32,
 17: -32,
 18: -39,
 19: -32}

In [22]:
quad_Y = [2 for (i, j) in comb_list] #異なるy同士

In [23]:
quad_SEX  = [2*SEX[i] * SEX[j] for (i, j) in comb_list]

In [24]:
quad_AOP = [2*AOP[i] * AOP[j] for (i, j) in comb_list]

In [26]:
quad_values = [sum(quad) for quad in zip(quad_Y, quad_SEX, quad_AOP)]
quad = {ij: quad_values[n] for (n, ij) in enumerate(comb_list)}

In [27]:
quad_values

[4,
 4,
 4,
 6,
 4,
 6,
 2,
 6,
 4,
 2,
 4,
 6,
 6,
 2,
 4,
 4,
 4,
 6,
 4,
 2,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 2,
 4,
 4,
 4,
 2,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 2,
 4,
 4,
 2,
 2,
 4,
 4,
 2,
 2,
 2,
 2,
 4,
 2,
 4,
 4,
 4,
 2,
 4,
 4,
 2,
 2,
 4,
 4,
 2,
 2,
 2,
 2,
 4,
 2,
 4,
 6,
 2,
 6,
 4,
 2,
 4,
 6,
 6,
 2,
 4,
 4,
 4,
 6,
 4,
 4,
 2,
 4,
 4,
 2,
 2,
 4,
 4,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 6,
 4,
 2,
 4,
 6,
 6,
 2,
 4,
 4,
 4,
 6,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 2,
 4,
 6,
 6,
 2,
 4,
 4,
 4,
 6,
 4,
 2,
 2,
 4,
 4,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 4,
 2,
 4,
 4,
 4,
 4,
 4,
 6,
 2,
 4,
 4,
 4,
 6,
 4,
 2,
 4,
 4,
 4,
 6,
 4,
 2,
 2,
 2,
 2,
 2,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4]

In [28]:
quad

{(0, 1): 4,
 (0, 2): 4,
 (0, 3): 4,
 (0, 4): 6,
 (0, 5): 4,
 (0, 6): 6,
 (0, 7): 2,
 (0, 8): 6,
 (0, 9): 4,
 (0, 10): 2,
 (0, 11): 4,
 (0, 12): 6,
 (0, 13): 6,
 (0, 14): 2,
 (0, 15): 4,
 (0, 16): 4,
 (0, 17): 4,
 (0, 18): 6,
 (0, 19): 4,
 (1, 2): 2,
 (1, 3): 2,
 (1, 4): 4,
 (1, 5): 2,
 (1, 6): 4,
 (1, 7): 2,
 (1, 8): 4,
 (1, 9): 2,
 (1, 10): 2,
 (1, 11): 4,
 (1, 12): 4,
 (1, 13): 4,
 (1, 14): 2,
 (1, 15): 4,
 (1, 16): 4,
 (1, 17): 4,
 (1, 18): 4,
 (1, 19): 4,
 (2, 3): 4,
 (2, 4): 4,
 (2, 5): 4,
 (2, 6): 4,
 (2, 7): 2,
 (2, 8): 4,
 (2, 9): 4,
 (2, 10): 2,
 (2, 11): 2,
 (2, 12): 4,
 (2, 13): 4,
 (2, 14): 2,
 (2, 15): 2,
 (2, 16): 2,
 (2, 17): 2,
 (2, 18): 4,
 (2, 19): 2,
 (3, 4): 4,
 (3, 5): 4,
 (3, 6): 4,
 (3, 7): 2,
 (3, 8): 4,
 (3, 9): 4,
 (3, 10): 2,
 (3, 11): 2,
 (3, 12): 4,
 (3, 13): 4,
 (3, 14): 2,
 (3, 15): 2,
 (3, 16): 2,
 (3, 17): 2,
 (3, 18): 4,
 (3, 19): 2,
 (4, 5): 4,
 (4, 6): 6,
 (4, 7): 2,
 (4, 8): 6,
 (4, 9): 4,
 (4, 10): 2,
 (4, 11): 4,
 (4, 12): 6,
 (4, 13): 6,
 (4, 14)

In [5]:
for bit in [20, 25, 30, 35, 40, 46]:
    if bit != 46:
        for num in [0, 1, 2, 3, 4]:
            path = '../input/ost{}_{}.csv'.format(bit, num)
            df = pd.read_csv(path, sep=',')
            res = net.make_res_data(df, num_reads)
            valid_y_info_dic = net.find_valid_y(res)
            plot_path = '{}bit/{}/nealSA_{}bit_{}_10000numreads.png'.format(bit, num, bit, num)
            hist_dic = net.y_num_hist(df, valid_y_info_dic, path)
    else:
        path = '../input/ost{}.csv'.format(bit)
        df = pd.read_csv(path, sep=',')
        res = net.make_res_data(df, num_reads)
        valid_y_info_dic = net.find_valid_y(res)
        plot_path = '{}bit/nealSA_{}bit_10000numreads.png'.format(bit, bit)
        hist_dic = net.y_num_hist(df, valid_y_info_dic, path)

In [6]:
for bit in [20, 25, 30, 35, 40, 46]:
    if bit != 46:
        for num in [0, 1, 2, 3, 4]:
            path = '../input/ost{}_{}.csv'.format(bit, num)
            df = pd.read_csv(path, sep=',')
            res = net.make_res_data(df, num_reads)
            valid_y_list, valid_y_num, occurrence_list = net.find_valid_y(res)
            plot_path = '{}bit/{}/nealSA_{}bit_{}_10000numreads_occurance_hist.png'.format(bit, num, bit, num)
            net.occurrence_hist(valid_y_info_dic, plot_path)
    else:
        path = '../input/ost{}.csv'.format(bit)
        df = pd.read_csv(path, sep=',')
        res = net.make_res_data(df, num_reads)
        valid_y_list, valid_y_num, occurrence_list = net.find_valid_y(res)
        plot_path = '{}bit/nealSA_{}bit_10000numreads_occurance_hist.png'.format(bit, bit)
        net.occurrence_hist(valid_y_info_dic, plot_path)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [ ]:
for bit in [20, 25, 30, 35, 40, 46]:
    if bit != 46:
        for num in [0, 1, 2, 3, 4]:
            path = '../input/ost{}_{}.csv'.format(bit, num)
            df = pd.read_csv(path, sep=',')
            path = '{}bit/{}/nealSA_{}bit_{}_10000numreads_time_num_hist.png'.format(bit, num, bit, num)
            time_list = net.time_num_y(df, num_reads, path)
    else:
        path = '../input/ost{}.csv'.format(bit)
        df = pd.read_csv(path, sep=',')
        path = '{}bit/nealSA_{}bit_10000numreads_time_num_hist.png'.format(bit, bit)
        time_list = net.time_num_y(df, num_reads, path)

In [ ]:
for bit in [20, 25, 30, 35, 40, 46]:
    if bit != 46:
        for num in [0, 1, 2, 3, 4]:
            path = '../input/ost{}_{}.csv'.format(bit, num)
            df = pd.read_csv(path, sep=',')
            res = net.make_res_data(df, num_reads)
            path = '{}bit/{}/nealSA_{}bit_{}_10000numreads_time_num_hist.png'.format(bit, num, bit, num)
            valid_y_list, p_dic = net.p_value_transition(res, path)
    else:
        path = '../input/ost{}.csv'.format(bit)
        df = pd.read_csv(path, sep=',')
        res = net.make_res_data(df, num_reads)
        path = '{}bit/nealSA_{}bit_10000numreads_time_num_hist.png'.format(bit, bit)
        valid_y_list, p_dic = net.p_value_transition(res, path)